# Finetuning Llama 3.2 Model into Embedding Model

## Goal

While LLaMA 3.2 is a powerful large language model (LLM) pre-trained on diverse datasets, its application to specific downstream tasks—such as semantic search, document retrieval, or natural language understanding—requires adapting the model to effectively generate dense vector representations (embeddings). In this tutorial, we will demonstrate how to finetune this model and convert it into a state-of-the-art embedding model for retrieval-augmented generation (RAG) tasks.

The key architectural change involves modifying the LLaMA model to optimize its performance in generating embeddings by replacing causal attention with bidirectional attention. This change enables the decoder-only model to create embeddings that are contextually relevant, semantically rich, and capable of improving the efficiency and accuracy of tasks like information retrieval, clustering, and text classification.

Our primary goals for this tutorial are as follows:

 * Demonstrate the ease of automatically converting the model with essential architectural changes for embedding model training
 * Improve the model's performance and accuracy in generating dense vector representations (embeddings)
 * Provide guidelines for finetuning embedding models, including hyperparameter choices.


# NeMo Tools and Resources

* [NeMo Framework](https://docs.nvidia.com/nemo-framework/user-guide/latest/overview.html)

# Software Requirements

* Access to latest NeMo Framework NGC Containers


# Hardware Requirements

* This playbook has been tested on the following hardware: Single A6000, Single H100, 2xA6000, 8xH100. It can be scaled to multiple GPUs as well as multiple nodes by modifying the appropriate parameters.


#### Launch the NeMo Framework container as follows: 

Depending on the number of gpus, `--gpus` might need to adjust accordingly:
```
docker run -it -p 8080:8080 -p 8088:8088 --rm --gpus '"device=0,1"' --ipc=host --network host -v $(pwd):/workspace nvcr.io/nvidia/nemo:25.02
```

#### Launch Jupyter Notebook as follows: 
```
jupyter notebook --allow-root --ip 0.0.0.0 --port 8088 --no-browser --NotebookApp.token=''

```

# Data

In this playbook, we will use the [Specter](https://huggingface.co/datasets/sentence-transformers/specter) datasets. 

The Specter dataset is a large collection of scientific papers and their abstracts, specifically designed for tasks like document classification, citation recommendation, and information retrieval. We will use the title-related-unrelated triplets from the dataset for finetuning. An example entry from this dataset is shown below:

```
Anchor: Millimeter-wave CMOS digital controlled artificial dielectric differential mode transmission lines for reconfigurable ICs
Positive: CMP network-on-chip overlaid with multi-band RF-interconnect
Negative: Route packets, not wires: on-chip interconnection networks
```

We will also demonstrate how to prepare and finetune the model using your own customized dataset

# Notebook Outline

* Step 1: Auto download and convert the original Llama 3.2 model to NeMo2
* Step 2: (Optional) Prepare your own dataset
* Step 3: Finetuning Llama 3.2 model to a embedding model
* Step 4: (Optional) Convert the model to HuggingFace Transformer format

# Step 1: Auto Download and Convert the Original Llama 3.2 Model to NeMo2

Llama 3.2 model can be automatically downloaded and converted to NeMo2 format with the following script:


In [ ]:
from nemo.collections import llm

if __name__ == '__main__':
    llm.import_ckpt(
        model=llm.LlamaEmbeddingModel(config=llm.Llama32EmbeddingConfig1B()),
        source="hf://meta-llama/Llama-3.2-1B",
    )

The above script 
- Downloads the LLaMA 3.2 1B model from Hugging Face (if not already downloaded).
- Automatically converts it into the NeMo format.
- Replaces all attention blocks with bidirectional attention, which plays a critical role in capturing the full context of a word or token in a sequence, as demonstrated by models like BERT (Bidirectional Encoder Representations from Transformers).

Note:
- The script can only run in a Python environment, not in a Jupyter notebook.
- Simply convert the above cell to a python script and run it with `python script.py`

The conversion will create a ```Llama-3.2-1B``` folder in the default ```$NEMO_HOME``` directory. 
```$NEMO_HOME``` centralizes and stores all models and datasets used for NeMo training. By default `$NEMO_HOME stores to ```~/.cache/nemo```.
This folder will be used in Step 3 for fine-tuning the model to generate the embedding model.

# Step 2: (Optional) Prepare Your own Dataset

If you want to use your own dataset for finetuning, prepare it as a single JSON file containing a list of JSON objects. Each object should include the following three fields:

* question: query or a prompt that the model needs to process and understand. In embedding-based models (especially in tasks like question answering (QA) or information retrieval), the "question" typically represents a user's query that seeks relevant information from a database or corpus of documents.
* pos_doc (Positive Document): "pos_doc" refers to a ```list``` of positive document — a document that is relevant to the given question. In other words, this document contains information that answers or is closely related to the question. 
* neg_doc (Negative Document): "neg_doc" refers to a ```list``` of negative document — a document that is irrelevant or less relevant to the question. This document either does not contain any useful information or contains information that does not answer or is only tangentially related to the question.

You can create the datamodule using NeMo's [CustomRetrievalDataModule](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/llm/gpt/data/retrieval.py#L31) to create the datamodule used for finetuning. ``CustomRetrievalDataModule`` takes one JSON `data_root` and performs train/val/test split, and it stores the processed dataset file to `$NEMO_HOME/datasets/<dataset_identifier>`. You can customize the ``query_key``, `pos_doc_key`, and `neg_doc_key` to match your own JSON dataset.

You can use [Specter](https://huggingface.co/datasets/sentence-transformers/specter/viewer/triplet) dataset as an example to better understand the question/pos_doc/neg_doc requires for embedding model training to prepare your own dataset.

In [ ]:
# JSON File contains a list of JSON objects with question, pos_doc, and neg_doc
[
    {
        "question": "What do floor indicaters often consist of?",
        "pos_doc": ["In addition to the call buttons, elevators usually have floor indicators (often illuminated by LED) and direction lanterns..."],
        "neg_doc": ["neg doc1", "neg doc2", "neg doc3", ...]
    },
    {
        "question": "An welchem Tag wurde das St. Helena Radio abgeschaltet?",
        "pos_doc": ["Radio St. Helena, die am Weihnachten 1967 begann, bot einen lokalen Radio-Service, der eine Reichweite von ca. 100 km (62 mi)..."],
        "neg_doc": ["neg doc1", "neg doc2", "neg doc3", ...]
    }
]

# Step 3: Finetuning Llama 3.2 model into an embedding model

For this step we use the NeMo2 predefined recipe. We will modify the recipe and showcase how you can adjust it to fit your own workflow. Typically, this involves changing the dataset, learning rate scheduler, and default parallelism etc.

First we define the recipe and executor for using NeMo2 as follows:


In [ ]:
import nemo_run as run
from nemo.collections import llm

def configure_recipe(nodes: int = 1, gpus_per_node: int = 1):
    recipe = llm.recipes.llama_embedding_1b.finetune_recipe(
        name="llama32_1b_embedding_finetuning",
        num_nodes=nodes,
        num_gpus_per_node=gpus_per_node,
    )
    return recipe

def local_executor_torchrun(devices: int = 1) -> run.LocalExecutor:
    executor = run.LocalExecutor(ntasks_per_node=devices, launcher="torchrun")
    return executor

# Instantiate the recipe
# Make sure you set the gpus_per_node as expected
recipe = configure_recipe(gpus_per_node=1) 

You can learn more about NeMo Executor [here](https://github.com/NVIDIA/NeMo-Run/blob/main/docs/source/guides/execution.md).

Let's understand a bit in depth of how the `recipe` works here:

`recipe` has few components: 
- model: the model to train
- data: the datamodule to train,
- trainer: Pytorch Lightning trainer that defines training strategy 
- log: logger for logging the training progress
- optim: optimizer to use for training,
- resume: resume connector to resume from a checkpoint prior training

The default `recipe`  initializes all the essential components required for finetuning the LLaMA 3.2 1B embedding model.

An important point to remember is that at this stage, you are only configuring your task; the underlying code is not executed yet.
This allows you to modify the configuration to fit your own custom training workflow.

In this task, `model` is initialized to `LlamaEmbeddingModel`, which, by default, uses ranking loss to learn the embedding.
Ranking loss penalizes the model based on how well it ranks the correct documents relative to incorrect ones. By default, the loss function only calculates the hard negatives provided by the dataset. However, you can treat examples within the batch as additional negatives by modifying the model's [config](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/llm/gpt/model/llama_embedding.py#L117):

In [ ]:
# (Optional) Change the loss objective to also take consideration of in-batch negatives:
recipe.model.config.in_batch_negatives = True

For this particular task of finetuning an embedding model, we want to adjust the recipe with smaller learning rate, and this can be achieved by:

In [ ]:
# (Optional) Modify the LR scheduler
recipe.optim.config.lr = 5e-6
recipe.optim.lr_scheduler.min_lr = 5e-7

Optionally, you can also change the parallelism settings, typically for the small 1B embedding model, tensor parallelism/ pipeline parallelism/ context parallelism are not required to hold the model:

In [ ]:
# (Optional) Modify the TP/PP/CP settings
# For small model (that fits well in single GPU), using TP1PP1CP1 typically yields the best performance.
recipe.trainer.strategy.tensor_model_parallel_size = 1
recipe.trainer.strategy.pipeline_model_parallel_size = 1
recipe.trainer.strategy.context_parallel_size = 1

You can also configure trainer-related setting such as decreasing the default training steps:

In [ ]:
# For the purpose of the demo, we limit the max_steps to 100
recipe.trainer.max_steps = 100
recipe.trainer.val_check_interval = 10
recipe.trainer.limit_val_batches = 5

Datamodule is swappable as well. The default `recipe` uses `SpecterDataModule`, if you want to use your customized dataset as described above, replace the data module as followed:



In [ ]:
def get_customize_dataloader(
  data_root='path/to/json/data',
  dataset_identifier='id_to_store_dataset',
  seq_length=512,
  micro_batch_size=16,
  global_batch_size=64,
  tokenizer=None,
  num_workers=16,
):
    return run.Config(
      llm.CustomRetrievalDataModule,
      data_root=data_root,
      dataset_identifier=dataset_identifier,
      seq_length=seq_length,
      micro_batch_size=micro_batch_size,
      global_batch_size=global_batch_size,
      tokenizer=tokenizer,
      num_workers=num_workers,
    )
# To override the datamodule, uncomment below
# recipe.data = dataloader

After configure the training procedure properly, we can run the training by instantiate the `executor` and use `nemorun` to start the training:

In [ ]:
executor = local_executor_torchrun(devices=recipe.trainer.devices)
run.run(recipe, executor=executor)

# Step 4: (Optional) Convert the Model to HuggingFace Transformer format

Use `llm.export_ckpt` to automatically convert the finetuned model into HuggingFace model. Similar to converting Hugging Face model to NeMo, you will need to run this in Python script as well:

In [ ]:
from pathlib import Path
llm.export_ckpt(
    path = Path('path/to/finetuned/ckpt'),
    target='hf',
    output_path=Path('path/to/converted/hf/ckpt'),
)